In [1]:
from __future__ import unicode_literals

import sys
import os
from bs4 import BeautifulSoup
import urllib2
import cv2
from matplotlib import pyplot as plt
import matplotlib
import threading
import time
import Queue
from PIL import Image, ImageDraw, ImageFont

In [2]:
# from https://stackoverflow.com/questions/4460921/extract-the-first-paragraph-from-a-wikipedia-article-python
import re
import textwrap
import yaml
import urllib
import urllib2
import random

import wikipedia

In [3]:
# set utf-8
stdi, stdo, stde = sys.stdin, sys.stdout, sys.stderr
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdin, sys.stdout, sys.stderr = stdi, stdo, stde
sys.getdefaultencoding()

'utf-8'


In [4]:
def de(s):
    print s

In [5]:
OUTPUT_PATH = '/home/ziweihe/sdb1/cvdata/wikipages500_2_kozh/'
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

In [6]:
DATA_SIZE = 500
IMG_SIZE = (227, 227)

In [7]:
LANG = {
    'ko': 'https://ko.wikipedia.org',
    'zh': 'https://zh.wikipedia.org'
}

In [8]:
class WikipediaError(Exception):
    pass

In [9]:
class Wikipedia:
    url_article = 'http://%s.wikipedia.org/w/index.php?action=raw&title=%s'
    url_image = 'http://%s.wikipedia.org/w/index.php?title=Special:FilePath&file=%s'
    url_search = 'http://%s.wikipedia.org/w/api.php?action=query&list=search&srsearch=%s&sroffset=%d&srlimit=%d&format=yaml'
   
    def __init__(self, lang):
        self.lang = lang
   
    def __fetch(self, url):
        request = urllib2.Request(url)
        request.add_header('User-Agent', 'Mozilla/5.0')
       
        try:
            result = urllib2.urlopen(request)
        except urllib2.HTTPError, e:
            raise WikipediaError(e.code)
        except urllib2.URLError, e:
            raise WikipediaError(e.reason)
       
        return result
   
    def article(self, article):
        url = self.url_article % (self.lang, urllib.quote_plus(article))
        content = self.__fetch(url).read()
       
        if content.upper().startswith('#REDIRECT'):
            match = re.match('(?i)#REDIRECT \[\[([^\[\]]+)\]\]', content)
           
            if not match == None:
                return self.article(match.group(1))
           
            raise WikipediaError('Can\'t found redirect article.')
       
        return content
   
    def image(self, image, thumb=None):
        url = self.url_image % (self.lang, image)
        result = self.__fetch(url)
        content = result.read()
       
        if thumb:
            url = result.geturl() + '/' + thumb + 'px-' + image
            url = url.replace('/commons/', '/commons/thumb/')
            url = url.replace('/' + self.lang + '/', '/' + self.lang + '/thumb/')
           
            return self.__fetch(url).read()
       
        return content
   
    def search(self, query, page=1, limit=10):
        offset = (page - 1) * limit
        url = self.url_search % (self.lang, urllib.quote_plus(query), offset, limit)
        content = self.__fetch(url).read()
       
        parsed = yaml.load(content)
        search = parsed['query']['search']
       
        results = []
       
        if search:
            for article in search:
                title = article['title'].strip()
               
                snippet = article['snippet']
                snippet = re.sub(r'(?m)<.*?>', '', snippet)
                snippet = re.sub(r'\s+', ' ', snippet)
                snippet = snippet.replace(' . ', '. ')
                snippet = snippet.replace(' , ', ', ')
                snippet = snippet.strip()
               
                wordcount = article['wordcount']
               
                results.append({
                    'title' : title,
                    'snippet' : snippet,
                    'wordcount' : wordcount
                })
       
        # yaml.dump(results, default_style='', default_flow_style=False,
        #     allow_unicode=True)
        return results

In [10]:
def unwiki(wiki):
    """
   Remove wiki markup from the text.
   """
    wiki = re.sub(r'(?i)\{\{IPA(\-[^\|\{\}]+)*?\|([^\|\{\}]+)(\|[^\{\}]+)*?\}\}', lambda m: m.group(2), wiki)
    wiki = re.sub(r'(?i)\{\{Lang(\-[^\|\{\}]+)*?\|([^\|\{\}]+)(\|[^\{\}]+)*?\}\}', lambda m: m.group(2), wiki)
    wiki = re.sub(r'\{\{[^\{\}]+\}\}', '', wiki)
    wiki = re.sub(r'(?m)\{\{[^\{\}]+\}\}', '', wiki)
    wiki = re.sub(r'(?m)\{\|[^\{\}]*?\|\}', '', wiki)
    wiki = re.sub(r'(?i)\[\[Category:[^\[\]]*?\]\]', '', wiki)
    wiki = re.sub(r'(?i)\[\[Image:[^\[\]]*?\]\]', '', wiki)
    wiki = re.sub(r'(?i)\[\[File:[^\[\]]*?\]\]', '', wiki)
    wiki = re.sub(r'\[\[[^\[\]]*?\|([^\[\]]*?)\]\]', lambda m: m.group(1), wiki)
    wiki = re.sub(r'\[\[([^\[\]]+?)\]\]', lambda m: m.group(1), wiki)
    wiki = re.sub(r'\[\[([^\[\]]+?)\]\]', '', wiki)
    wiki = re.sub(r'(?i)File:[^\[\]]*?', '', wiki)
    wiki = re.sub(r'\[[^\[\]]*? ([^\[\]]*?)\]', lambda m: m.group(1), wiki)
    wiki = re.sub(r"''+", '', wiki)
    wiki = re.sub(r'(?m)^\*$', '', wiki)

    return wiki

def unhtml(html):
    """
   Remove HTML from the text.
   """
    html = re.sub(r'(?i)&nbsp;', ' ', html)
    html = re.sub(r'(?i)<br[ \\]*?>', '\n', html)
    html = re.sub(r'(?m)<!--.*?--\s*>', '', html)
    html = re.sub(r'(?i)<ref[^>]*>[^>]*<\/ ?ref>', '', html)
    html = re.sub(r'(?m)<.*?>', '', html)
    html = re.sub(r'(?i)&amp;', '&', html)

    return html

def punctuate(text):
    """
   Convert every text part into well-formed one-space
   separate paragraph.
   """
    text = re.sub(r'\r\n|\n|\r', '\n', text)
    text = re.sub(r'\n\n+', '\n\n', text)

    parts = text.split('\n\n')
    partsParsed = []

    for part in parts:
        part = part.strip()

        if len(part) == 0:
            continue

        partsParsed.append(part)

    return '\n\n'.join(partsParsed)


In [11]:
# class worker_thread (threading.Thread):
#     def __init__(self, threadID, name, func, args):
#         threading.Thread.__init__(self)
#         self.threadID = threadID
#         self.name = name
#         self.func = func
#         self.args = args
#         self._stop_event = threading.Event()
        
#     def run(self):
#         de("Starting " + self.name)
#         self.func(self.threadID, self.args)
#         de("Exiting " + self.name)
        
#     def stop(self):
#         self._stop_event.set()

#     def stopped(self):
#         return self._stop_event.is_set()

In [12]:
atc_set = set([])
atc_lock = threading.Lock()
link_set = set([])
link_lock = threading.Lock()

threads = []

In [13]:
# def get_urls_task(id, url):
#     '''get urls in url and add to lists'''
#     try:
#         content = urllib.urlopen(url)
#     except:
#         return
#     soup = BeautifulSoup(content, "html.parser")
#     base=soup.find('div', id="bodyContent")
    
#     for prefix_key in LANG:
#         if LANG[prefix_key] in url:
#             prefix = LANG[prefix_key]

#     for link in BeautifulSoup(str(base), "html.parser").findAll("a"):
#         if 'href' in link.attrs:  # Make sure it is link
#             if link['href'].startswith('/wiki/') and prefix + link['href'] != url:
#                 link_lock.acquire()
#                 link_set.add(prefix + link['href'])
#                 link_lock.release()
#             if ( 
#                 link['href'].startswith('/wiki/') and  # Make sure it is a wiki page
#                 len(link['href'].split('/')) == 3 and  # Make sure it is an article
#                 len(link['href'].split(':')) == 1      # Make sure it is not category
#             ):
# #                 print(link['href'])
#                 title = str(link['href'])
#                 title = urllib.unquote(title).decode('utf8')
#                 print title
#                 atc_lock.acquire()
#                 if len(atc_set) < DATA_SIZE:
#                     atc_set.add(prefix + title)
#                 atc_lock.release()

In [14]:
# link_set.add(LANG['en'])
# thread_id = 1
# while len(atc_set) < DATA_SIZE:
#     while len(link_set) == 0:
#         pass
#     url = link_set.pop()
#     thread = worker_thread(thread_id, url, get_urls_task, url)
#     thread.start()
#     threads.append(thread)
#     thread_id += 1
#     time.sleep(1)

In [15]:
# link_set_en = link_set
# link_set = set([])

# atc_set_en = atc_set
# atc_set = set([])

In [16]:
# link_set.add(LANG['zh'])
# thread_id = 1
# while len(atc_set) < DATA_SIZE:
#     while len(link_set) == 0:
#         pass
#     url = link_set.pop()
#     thread = worker_thread(thread_id, url, get_urls_task, url)
#     thread.start()
#     threads.append(thread)
#     thread_id += 1
#     time.sleep(1)

In [17]:
# for i in atc_set_en:
#     atc_set.add(i)
# print len(atc_set)

In [18]:
def article_urls_crawl(prefix, root_url, count):
    atc_list = set([])
    link_list = set([root_url])
    while True:
        url = random.choice(tuple(link_list))
        try:
            content = urllib.urlopen(url)
        except:
            continue
#         print urllib.urlopen(url).headers.getheader('Content-Type')
        soup = BeautifulSoup(content, "html.parser")
        base=soup.find('div', id="bodyContent")

        for link in BeautifulSoup(str(base), "html.parser").findAll("a"):
            if 'href' in link.attrs:  # Make sure it is link
                if link['href'].startswith('/wiki/') and prefix + link['href'] != url:
                    link_list.add(prefix + link['href'])
                if ( 
                    link['href'].startswith('/wiki/') and  # Make sure it is a wiki page
                    len(link['href'].split('/')) == 3 and  # Make sure it is an article
                    len(link['href'].split(':')) == 1      # Make sure it is not category
                ):
#                     print(link['href'])
                    title = str(link['href'])
                    title = urllib.unquote(title).decode('utf8')
                    print title
                    atc_list.add(prefix + title)
                    if len(atc_list) == count:
                        print('fetched ' + str(count) + ' article urls from ' + prefix + '\n')
                        return atc_list

urls_by_lang = {}

for key in LANG:
    urls_by_lang[key] = article_urls_crawl(LANG[key], LANG[key], DATA_SIZE)

/wiki/自由內容
/wiki/天津历史文化街区列表
/wiki/天津市
/wiki/天津歷史
/wiki/天津租界
/wiki/天津市区
/wiki/法国
/wiki/意大利
/wiki/英国
/wiki/周朝
/wiki/秦朝
/wiki/新朝
/wiki/联合国地理区划列表
/wiki/印度行政区人口列表
/wiki/印度各邦和中央直辖区首府列表
/wiki/台灣
/wiki/澎湖
/wiki/風櫃威武金王殿
/wiki/萬善爺
/wiki/美國總統
/wiki/唐納德·特朗普
/wiki/英國
/wiki/倫敦
/wiki/愛丁堡
/wiki/唐納德·特朗普寶寶氣球
/wiki/李季伦
/wiki/中国农业大学
/wiki/石元春
/wiki/2018年颶風赫克托
/wiki/颶風吉納維芙_(2014年)
/wiki/熱帶氣旋
/wiki/魏晋南北朝
/wiki/子夜歌
/wiki/晋朝
/wiki/台灣
/wiki/澎湖
/wiki/吳克文
/wiki/瓦爾特·馮·賴歇瑙
/wiki/納粹德國陸軍
/wiki/德国元帅
/wiki/第二次世界大战
/wiki/阿道夫·希特勒
/wiki/纳粹党
/wiki/冲锋队_(纳粹党)
/wiki/军_(军队)
/wiki/軍團
/wiki/集团军
/wiki/苏德战争
/wiki/乌克兰族
/wiki/白俄罗斯人
/wiki/格特·馮·倫德施泰特
/wiki/南方集團軍
/wiki/心脏病
/wiki/让-米歇尔·雅尔
/wiki/法国
/wiki/里昂
/wiki/電子音樂
/wiki/2018年中華民國邦交國斷交與代表機構更名事件
/wiki/中華民國與薩爾瓦多關係
/wiki/中国－萨尔瓦多关系
/wiki/懶人小鎮
/wiki/斯特凡·卡爾·斯蒂芬森
/wiki/穆罕默德六世_(摩洛哥)
/wiki/徵兵制
/wiki/摩洛哥议会
/wiki/尼古拉斯·马杜罗
/wiki/委內瑞拉玻利瓦
/wiki/石油币
/wiki/喀拉拉邦
/wiki/2018年北印度洋氣旋季
/wiki/2018年8月逝世人物列表
/wiki/薩杜·哈亞圖
/wiki/科菲·安南
/wiki/鮑勃·巴斯
/wiki/艾瑞莎·弗蘭克林
/wiki/8月24日
/wiki/乌克兰
/wiki/独立日
/wiki/1991年
/wiki

/wiki/프랑스
/wiki/리옹
/wiki/전자_음악
/wiki/장미셸_자르
/wiki/위키미디어_공용
/wiki/2013년_세계_육상_선수권_대회
/wiki/우사인_볼트
/wiki/살라르_데_우유니
/wiki/토요타_랜드크루저
/wiki/애틀랜티스_우주왕복선
/wiki/인데버_우주왕복선
/wiki/케네디_국제공항
/wiki/혜성
/wiki/체코
/wiki/프라하
/wiki/카를교
/wiki/프랑스
/wiki/리옹
/wiki/전자_음악
/wiki/장미셸_자르
/wiki/재규어
/wiki/프랑스_(동음이의)
/wiki/프랑스의_국기
/wiki/프랑스의_국장
/wiki/자유,_평등,_우애
/wiki/국가_목록
/wiki/프랑스의_국가
/wiki/수도
/wiki/파리_(프랑스)
/wiki/파리_(프랑스)
/wiki/정치
/wiki/공용어_목록
/wiki/프랑스어
/wiki/바스크어
/wiki/정치체제
/wiki/공화제
/wiki/민주제
/wiki/입헌제
/wiki/단일제
/wiki/이원집정부제
/wiki/프랑스의_대통령
/wiki/에마뉘엘_마크롱
/wiki/프랑스의_총리
/wiki/에두아르_필리프
/wiki/프랑스의_역사
/wiki/843년
/wiki/베르됭_조약
/wiki/프랑스_혁명
/wiki/1789년
/wiki/7월_14일
/wiki/1958년
/wiki/10월_4일
/wiki/지리
/wiki/넓이
/wiki/면적순_나라_목록
/wiki/시간대
/wiki/중앙유럽_표준시
/wiki/협정_세계시
/wiki/일광_절약_시간제
/wiki/중앙유럽_일광_절약_시간대
/wiki/인구순_나라_목록
/wiki/2016년
/wiki/2010년
/wiki/인구_조사
/wiki/인구_밀도
/wiki/인구_밀도순_나라_목록
/wiki/경제
/wiki/국내_총생산
/wiki/구매력_평가
/wiki/구매력평가_기준_국내총생산순_나라_목록
/wiki/구매력평가_기준_일인당_국내총생산순_나라_목록
/wiki/인간_개발_지수
/wiki/인간_개발_지수순_나라_목록
/wiki/2016년

In [19]:
def getChinese(context):
    context = context.decode("utf-8") # convert context from str to unicode
    filtrate = re.compile(u'[^\u4E00-\u9FA5]') # non-Chinese unicode range
    context = filtrate.sub(r'', context) # remove all non-Chinese characters
    context = context.encode("utf-8") # convert unicode back to str
    return context

In [20]:
print "Заглавная_страница"

Заглавная_страница


In [21]:
# fontsizerange = (15, 50)
# def gen_img_task(id, url):
#     de("generating image for " + url)
#     for prefix_key in LANG:
#         if LANG[prefix_key] in url:
#             key = prefix_key
    
#     if key is None:
#         return
    
#     de("detected language " + key)
    
#     wikipedia.set_lang(key)
#     res = None
#     try:
#         res = wikipedia.summary(url.split('/')[-1])
#     except:
#         return

# #     text = res
#     text = ""
#     if key == 'en':
#         for n in re.findall(ur'[a-zA-Z ]+',res):
#             text = text + n
#     elif key == 'zh':
#         for n in re.findall(ur'[\u4e00-\u9fff]+',res):
#             text = text + n
# #     de("printing " + text)

#     fontsize = random.randint(fontsizerange[0], fontsizerange[1])
#     fontwidth = IMG_SIZE[1]/(fontsize/3*4)*3

#     image = Image.new('L', IMG_SIZE, 255)
#     draw = ImageDraw.Draw(image)
#     if key == 'en':
#         font = ImageFont.truetype('/usr/share/fonts/dejavu/DejaVuSans.ttf', fontsize)
#         margin = 0
#         offset = 0
#         for line in textwrap.wrap(text, width=fontwidth):
#             draw.text((margin, offset), line, font=font, fill="#000000")
#             offset += font.getsize(line)[1]
#     elif key == 'zh':
#         font = ImageFont.truetype('/home/ziweihe/fonts/msyh.ttf', fontsize)
#         margin = 0
#         offset = 0
#         for line in textwrap.wrap(text, width=fontwidth):
#             draw.text((margin, offset), line, font=font, fill="#000000")
#             offset += font.getsize(line)[1]
#     de("saving " + OUTPUT_PATH + key + '-' + str(id) + '-wiki.jpg')
#     image.save(OUTPUT_PATH + key + '-' + str(id) + '-wiki.jpg')

In [22]:
# thread_id = 1
# while len(atc_set) > 0:
#     url = atc_set.pop()
#     thread = worker_thread(thread_id, url, gen_img_task, url)
#     thread.start()
#     threads.append(thread)
#     thread_id += 1

In [23]:
def generate_imgs(langs, urls_by_lang, count, fontsizerange):
    for key in langs:
        print 'generating imgs for ' + key
        i = 0
        while i < count:
            wikipedia.set_lang(key)
            res = None
            try:
                res = wikipedia.summary(random.choice(tuple(urls_by_lang[key])).split('/')[-1])
            except:
                continue
            
            text = ""
            if key == 'ko':
                for n in re.findall(ur'[\u3131-\ucb4c]+',res):
                    text = text + n
            elif key == 'zh':
                for n in re.findall(ur'[\u4e00-\u9fff]+',res):
                    text = text + n
#             print text


            fontsize = random.randint(fontsizerange[0], fontsizerange[1])
            fontwidth = IMG_SIZE[1]/(fontsize/3*4)*3

            image = Image.new('L', IMG_SIZE, 255)
            draw = ImageDraw.Draw(image)
            if key == 'ko':
                font = ImageFont.truetype('/home/ziweihe/fonts/NanumSquareB.ttf', random.randint(fontsizerange[0], fontsizerange[1]))
                margin = 0
                offset = 0
                for line in textwrap.wrap(text, width=fontwidth):
                    draw.text((margin, offset), line, font=font, fill="#000000")
                    offset += font.getsize(line)[1]
            elif key == 'zh':
                font = ImageFont.truetype('/home/ziweihe/fonts/msyh.ttf', random.randint(fontsizerange[0], fontsizerange[1]))
                margin = 0
                offset = 0
                for line in textwrap.wrap(text, width=fontwidth):
                    draw.text((margin, offset), line, font=font, fill="#000000")
                    offset += font.getsize(line)[1]
            image.save(OUTPUT_PATH + key + '-' + str(i) + '-wiki.jpg')
            i += 1

In [24]:
generate_imgs(LANG, urls_by_lang, DATA_SIZE, (60, 60))

generating imgs for zh


/home/ziweihe/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /home/ziweihe/anaconda2/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


generating imgs for ko
